# Ad-hoc RAG

Evaluate results from an ad-hoc RAG similar than the one behind franceservices.etalab.gouv.fr using evalap and ModelRaw schema

In [ ]:
import os
import sys
import time
from io import StringIO
import concurrent.futures

import dotenv
from IPython.display import HTML
import numpy as np
import pandas as pd
import requests
from jinja2 import Template

dotenv.load_dotenv("../.env")
sys.path.append("..")
from evalap.utils import log_and_raise_for_status

EVALAP_API_URL = "http://localhost:8000/v1"
#EVALAP_API_URL = "https://evalap.etalab.gouv.fr/v1"
EVALAP_API_KEY = os.getenv("EVALAP_API_KEY") 
ALBERT_API_URL = "https://albert.api.etalab.gouv.fr/v1"
ALBERT_API_KEY = os.getenv("ALBERT_API_KEY")
OPENAI_URL = "https://api.openai.com/v1"
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
headers = {"Authorization": f"Bearer {EVALAP_API_KEY}"}

In [ ]:
# Get the dataset
dataset_name = "MFS_questions_v01"
response = requests.get(
    f"{EVALAP_API_URL}/dataset?name={dataset_name}&with_df=true",
    headers={"Authorization": f"Bearer {EVALAP_API_KEY}"},
)
response.raise_for_status()
dataset = response.json()
dataset_df =  pd.read_json(StringIO(dataset["df"]))

In [ ]:
sys.path.append("../../mcp-servers/data-gouv-fr/")
from clients.search import SearchEngineClient
from clients.llm import LlmClient, split_think_answer

In [ ]:
sampling_params = {"temperature": 0.2}
collection_name = "chunks-v6"
model_embedding = "BAAI/bge-m3"
limit = 10

system_prompt = "Tu es un agent de l'état Français qui répond en langue française aux questions des usagers des services publiques et citoyens de manière précise, concrète et concise."
rag_prompt = Template("""Ecris un texte référencé en réponse à cette question : {{query}}

Les références doivent être citées de cette manière (échape les guillements avec \\", s'il y en a, dans les attributs de <ref>) : texte rédigé <ref title="[titre de la source]" text="[passage pertinent dans la référence]">[URL de la source]</ref>

Si les références ne permettent pas de répondre, spécifie juste qu'il n'y a pas de réponse.

Les {{limit}} références disponibles :
{% for chunk in chunks %}
url: {{chunk.url}}
title: {{chunk.title}} {% if chunk.context %}({{chunk.context}}){% endif %}
text: {{chunk.text}} {% if not loop.last %}{{"\n"}}{% endif %}
{% endfor %}
""")

# Augment a prompt with collection search
def do_rag(query, limit=limit):
    # Search relevant chunks
    se_config = dict(
        es_url=os.getenv("ELASTICSEARCH_URL"),
        es_creds=("elastic", os.getenv("ELASTICSEARCH_PASSWORD")),
        model_embedding=model_embedding,
    )
    se_client = SearchEngineClient(**se_config)
    hits = se_client.search(collection_name, query, limit=limit)

    # Render prompt
    return rag_prompt.render(query=query, chunks=hits, limit=limit)

# The LLM core generation
def generate(model, prompt, use_system_prompt=True):
    messages = [{"role": "user", "content": prompt}]
    if use_system_prompt:
        messages = [{"role": "system", "content": system_prompt}] + messages
        
    aiclient = LlmClient()
    result = aiclient.generate(model=model, messages=messages, **sampling_params)
    observation = result.choices[0].message.content
    think, answer = split_think_answer(observation)
    return answer

In [ ]:
print(do_rag("CNI", limit=3))

In [ ]:
# Async computing
# --

# The models to runs
models = ["AgentPublic/llama3-instruct-guillaumetell", "meta-llama/Llama-3.1-8B-Instruct", "mistralai/Mistral-Small-3.1-24B-Instruct-2503"]

# the input prompts to answer
prompts = []
for i, row in dataset_df.iterrows():
    prompts.append(do_rag(row["query"]))

# Loop over the model to try
model_raws = []
for model in models:
    # Async over the prompts
    with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
        # Create a list of model arguments (same model repeated for each prompt)
        model_args = [model] * len(prompts)

        # Map generate over pairs of (model, prompt)
        results = list(executor.map(generate, model_args, prompts))

    model_raws.append(results)

In [ ]:
judge_name = "gpt-4.1"
judge_api_url = "https://api.openai.com/v1",
judge_api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
# Build the expset
# --
expset_name = "MFS ad-hoc RAG"
expset_readme = "MFS ad-hoc RAG (similar parametriation than expset 50)"
common_params = {
    "dataset": dataset["name"],
    "metrics": ["judge_precision", "output_length"],
    "judge_model": {
        "name": judge_name,
        "base_url": judge_api_url,
        "api_key": judge_api_key,
},
}

grid_params = {
    "model": [
        {
            "aliased_name": models[i].split("/")[-1] + "-adhoc-rag",
            "name": models[i],
            "system_prompt": system_prompt,
            "sampling_params": sampling_params,
            "output": outputs,
        }
        for i, outputs in enumerate(model_raws)
    ],
}

expset = {
    "name": expset_name,
    "readme": expset_readme,
    "cv": {"common_params": common_params, "grid_params": grid_params, "repeat": 1},
}

response = requests.post(
    f"{EVALAP_API_URL}/experiment_set",
    headers={"Authorization": f"Bearer {EVALAP_API_KEY}", "Content-Encoding": "gzip"},
    json=expset,
)
resp = response.json()
if "id" in resp:
    expset_id = resp["id"]
    print(f'Created expset: {resp["name"]} ({resp["id"]})')
else:
    print(resp)